In [1]:
# :: 29th January 2023

In [2]:
import librosa, json, os, numpy as np, matplotlib.pyplot as plt, torch, tensorflow.keras as keras
from sklearn.model_selection import train_test_split

DATASET_PATH = 'speech_commands_v0.01'
DATA_PATH = 'data.json'
MINIMUM_SAMPLES = 22050 # 1 second
SAVED_MODEL_PATH = 'model.pth'
PATIENCE = 5
BATCH_SIZE = 32
EPOCHS = 5

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs

        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()

In [ ]:
def process_dataset_to_json(dataset_path, json_path, n_mfcc=13, hop_length=512, n_fft=2048):
    data = {
        'mappings': [], # 
        'labels': [], # These values are the target values that we expect at index i.
        'MFCCs': [], # Store the MFCC's for each audio clip.
        'files': [] # Want file path of audio clip, to be able to go back and do analysis.
    }
    
    # Go through dataset sub directories
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        # Check we are not at dataset level
        if dirpath is not dataset_path:
            # Update mappings
            category_name = dirpath.split('/') # dataset/down -> [dataset, down] where category_name = down
            data['mappings'].append(category_name)
            
            print(f'Processing category: {category_name}')


            # Looping through filenames and extracting MFCCs
            for f in filenames:
                # Get file path.
                file_path = os.path.join(dirpath, f)
                # Load audio file.
                signal, sr = librosa.load(file_path)
                # Ensure audio file is >= 1 second.
                if len(signal) >= MINIMUM_SAMPLES:
                    # Ensure 1 second sample
                    signal = signal[:MINIMUM_SAMPLES]
                    # Extract MFCC
                    mfcc = librosa.feature.mfcc(signal, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
                    # Store data
                    data['labels'].append(i-1) # Count - 1
                    data['MFCCs'].append(mfcc.T.tolist()) # We cannot store ndarrays in JSON so we cast to list
                    data['files'].append(file_path)
                    
                    print(f'{file_path} {i-1}')
                    
    with open(json_path, 'w') as fp:
        json.dump(data, fp, indent=4)
                

In [ ]:
# Generate json file
# process_dataset_to_json(DATASET_PATH, JSON_PATH)

In [ ]:
def load_data(data_path):
    with open(data_path, 'r') as fp:
        data = json.load(fp)
    
    X = np.array(data['MFCCs'])
    y = np.array(data["labels"])
    print("Training sets loaded.")
    return X, y

In [ ]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [ ]:
def build_model(input_shape):
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(31, activation='softmax'))

    return model

In [ ]:
def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))


In [ ]:
# get train, validation, test splits
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

# create network
input_shape = (X_train.shape[1], X_train.shape[2], 1)
model = build_model(input_shape)

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=1)

# plot accuracy/error for training and validation
plot_history(history)

# evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# pick a sample to predict from the test set
X_to_predict = X_test[100]
y_to_predict = y_test[100]

# predict sample
predict(model, X_to_predict, y_to_predict)

In [ ]:
# pick a sample to predict from the test set
X_to_predict = X_test[14]
y_to_predict = y_test[14]

# predict sample
predict(model, X_to_predict, y_to_predict)

Target: 11, Predicted label: [5]
